In [2]:
from dotenv import load_dotenv

load_dotenv()

import os

In [3]:
from openai import OpenAI
client = OpenAI()

In [4]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="tell me a joke",
)
print(response.output_text)

second_response = client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=response.id,
    input=[{"role": "system", "content": "you joke about the weather"}],
)
print(second_response.output_text)



KeyboardInterrupt: 

In [ ]:
third_response = client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=second_response.id,
    input=[{"role": "user", "content": "what was my first question?"}],
)
print(third_response.output_text)


Your first question was, "tell me a joke."


In [22]:
client.responses.input_items.list(third_response.id)

SyncCursorPage[Annotated[Union[ResponseInputMessageItem, ResponseOutputMessage, ResponseFileSearchToolCall, ResponseComputerToolCall, ResponseComputerToolCallOutputItem, ResponseFunctionWebSearch, ResponseFunctionToolCallItem, ResponseFunctionToolCallOutputItem], PropertyInfo]](data=[ResponseInputMessageItem(id='msg_67fd37d350a08192a11de8eb358253c8050c94c450012a2d', content=[ResponseInputText(text='what was my first question?', type='input_text')], role='user', status='completed', type='message'), ResponseOutputMessage(id='msg_67fd3786368081929b4172d4615c1d52050c94c450012a2d', content=[ResponseOutputText(annotations=[], text='This joke is funny for a couple of reasons:\n\n1. **Wordplay**: The phrase "don’t have the guts" is a play on words. Literally, it refers to skeletons not having internal organs (like guts), but figuratively it means lacking bravery or courage.\n\n2. **Surprise Element**: The punchline subverts expectations. When you start with a statement about skeletons fighting

In [30]:
fetched_response = client.responses.retrieve(
response_id=third_response.id)

print(fetched_response)

Response(id='resp_67fd37d350188192a10eac5254b582ea050c94c450012a2d', created_at=1744648147.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_67fd37d3cbf081928cbb184086d6a7d1050c94c450012a2d', content=[ResponseOutputText(annotations=[], text='Your first question was, "tell me a joke."', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id='resp_67fd3785d5888192a66be229ed1f5109050c94c450012a2d', reasoning=Reasoning(effort=None, generate_summary=None), status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=224, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=12, output_tokens_details=OutputTokensDetails(reasoning_tokens=

In [ ]:
def retrieve_chat_history(last_response_id: str, simplified: bool = True):
    messages = []
    for item in client.responses.input_items.list(last_response_id).data:
        messages.append(item)
    messages.append(client.responses.retrieve(response_id=last_response_id).output[0])

    if simplified:
        dict_messages = []
        for item in messages:
            if item.type == 'message':
                dict_messages.append({"role": item.role, "content": item.content[0].text})
            elif item.type == 'function_call_output':
                dict_messages.append({"role": "tool", "content": item.output})
        return dict_messages
    else:
        return messages

retrieve_chat_history('resp_67fd406972bc8192ac0e6f2d86966cf0050c94c450012a2d')

[{'role': 'system', 'content': 'you joke about the weather'},
 {'role': 'assistant',
  'content': "Why don't skeletons fight each other? \n\nBecause they don't have the guts!"},
 {'role': 'user', 'content': 'tell me a joke'},
 {'role': 'assistant',
  'content': 'Why did the weather report bring a ladder? \n\nBecause it predicted high winds!'}]

In [54]:
messages = []

for item in client.responses.input_items.list(third_response.id).data:
    messages.append(item)

messages.append(client.responses.retrieve(response_id=third_response.id).output[0])

messages

[ResponseInputMessageItem(id='msg_67fd37d350a08192a11de8eb358253c8050c94c450012a2d', content=[ResponseInputText(text='what was my first question?', type='input_text')], role='user', status='completed', type='message'),
 ResponseOutputMessage(id='msg_67fd3786368081929b4172d4615c1d52050c94c450012a2d', content=[ResponseOutputText(annotations=[], text='This joke is funny for a couple of reasons:\n\n1. **Wordplay**: The phrase "don’t have the guts" is a play on words. Literally, it refers to skeletons not having internal organs (like guts), but figuratively it means lacking bravery or courage.\n\n2. **Surprise Element**: The punchline subverts expectations. When you start with a statement about skeletons fighting, you might expect a more serious reason why they wouldn\'t. The sudden switch to a light-hearted observation about their lack of guts creates humor.\n\n3. **Absurdity**: The image of skeletons fighting is inherently absurd, which adds to the humor. It’s funny to imagine something t

In [56]:
client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=response.id,
    input=[{"role": "system", "content": "you joke about the weather"}],
)

Response(id='resp_67fd406972bc8192ac0e6f2d86966cf0050c94c450012a2d', created_at=1744650345.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_67fd406a2c088192b3ae72d44e5f5679050c94c450012a2d', content=[ResponseOutputText(annotations=[], text='Why did the weather report bring a ladder? \n\nBecause it predicted high winds!', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id='resp_67fd3784be908192907643aebb25990c050c94c450012a2d', reasoning=Reasoning(effort=None, generate_summary=None), status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=40, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=17, output_tokens_details=O

In [67]:
import json

In [75]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
},
{
    "type": "function",
    "name": "send_email",
    "description": "Send an email to the specified recipient.",
    "parameters": {
        "type": "object",
        "properties": {
            "recipient": {"type": "string"},
            "subject": {"type": "string"},
            "body": {"type": "string"}
        },
        "required": ["recipient", "subject", "body"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="gpt-4o-mini",
    input=input_messages,
    tools=tools,
)

In [71]:
response.output[0].type

'function_call'

In [68]:

args = json.loads(response.output[0].arguments)
print(args)

{'latitude': 48.8566, 'longitude': 2.3522}


In [69]:
retrieve_chat_history(response.id)

[{'role': 'user', 'content': "What's the weather like in Paris today?"}]

In [232]:
from typing import Any, Dict, List, Optional

from openai import OpenAI

def call_function(name, args):
    if name == "get_weather":
        return "weather is sunny"
    if name == "send_email":
        return "email sent"

class FunctionCallingAgent:
    def __init__(
        self,
        model: str = "gpt-4o-mini",
        temperature: float = 1.0,
        tools: Optional[List[Dict[str, Any]]] = None,
        instructions: Optional[str] = None,
        previous_response_id: Optional[str] = None,
        max_tool_calls: int = 6,
    ):
        """Initialize the function calling agent.

        Args:
            model: The OpenAI model to use
            temperature: Sampling temperature
            tools: List of tools/functions available to the agent
        """
        self.client = OpenAI()

        self.model = model
        self.temperature = temperature
        self.tools = tools
        self.instructions = instructions
        self.previous_response_id = previous_response_id
        
        self.max_tool_calls = max_tool_calls

    def _handle_tool_calls(self, tool_calls) -> Any:
        tool_results = []
        for tool_call in tool_calls:
            if tool_call.type != "function_call":
                continue

            name = tool_call.name
            args = json.loads(tool_call.arguments)
        
            result = call_function(name, args)
            print(f"Function {name} called with args {args} and result {result}")

            tool_results.append({
                "type": "function_call_output",
                "call_id": tool_call.call_id,
                "output": str(result)
            })

        return tool_results

    def send_message(self, message: Optional[Dict[str, Any]] = None, **kwargs):

        # resolve kwargs with default values
        params = {
            "previous_response_id": self.previous_response_id,
            "model": self.model,
            "temperature": self.temperature,
            "tools": self.tools,
            "instructions": self.instructions,
        }
        params.update(kwargs)

        messages = []
        if message:
            messages.append(message)
        
        tool_calls_remaining = self.max_tool_calls
        
        while tool_calls_remaining > 0:
            response = self.client.responses.create(
                input=messages,
                **params
            )
            messages.extend(response.output)

            if not response.output or response.output[0].type != 'function_call':
                break
            
            tool_results = self._handle_tool_calls(response.output)
            messages.extend(tool_results)
            
            tool_calls_remaining -= 1

        self.last_response_id = response.id

        print(response.output[0].content[0].text)
        
        return response

In [233]:
def retrieve_chat_history(last_response_id: str, simplified: bool = True):
    messages = []

    system_message = {"role": "system", "content": client.responses.retrieve(last_response_id).instructions}

    for item in client.responses.input_items.list(last_response_id, limit=100).data:
        messages.append(item)
    messages = messages[::-1]
    messages.extend(client.responses.retrieve(response_id=last_response_id).output)

    if simplified:
        dict_messages = []
        for item in messages:
            if item.type == 'message':
                dict_messages.append({"role": item.role, "content": item.content[0].text, "type": "message"})
            elif item.type == 'function_call':
                dict_messages.append({"type": "function_call", "name": item.name, "arguments": item.arguments, "call_id": item.call_id})
            elif item.type == 'function_call_output':
                dict_messages.append({"type": "function_call_output", "output": item.output, "call_id": item.call_id})
        return [system_message] + dict_messages
    else:
        return [system_message] + messages

In [234]:
agent = FunctionCallingAgent()

In [244]:
from pydantic import BaseModel
import openai

class GetWeatherArgs(BaseModel):
    """Get the current weather in a given location."""
    latitude: float
    longitude: float

class SendEmailArgs(BaseModel):
    recipient: str
    subject: str
    body: str



def generate_schema(model, name):
    _schema = openai.pydantic_function_tool(model, name=name)
    return {
        "type": "function",
        **_schema['function']
    }

pydantic_tools = [generate_schema(GetWeatherArgs, "get_weather"), generate_schema(SendEmailArgs, "send_email")]
pydantic_tools

[{'type': 'function',
  'name': 'get_weather',
  'strict': True,
  'parameters': {'description': 'Get the current weather in a given location.',
   'properties': {'latitude': {'title': 'Latitude', 'type': 'number'},
    'longitude': {'title': 'Longitude', 'type': 'number'}},
   'required': ['latitude', 'longitude'],
   'title': 'GetWeatherArgs',
   'type': 'object',
   'additionalProperties': False},
  'description': 'Get the current weather in a given location.'},
 {'type': 'function',
  'name': 'send_email',
  'strict': True,
  'parameters': {'properties': {'recipient': {'title': 'Recipient',
     'type': 'string'},
    'subject': {'title': 'Subject', 'type': 'string'},
    'body': {'title': 'Body', 'type': 'string'}},
   'required': ['recipient', 'subject', 'body'],
   'title': 'SendEmailArgs',
   'type': 'object',
   'additionalProperties': False}}]

In [245]:
response = agent.send_message(
    {"role": "user", "content": "Get the current weather in Paris and Delhi and then send an email to john@example.com with the weather report."},
    model="gpt-4o-mini",
    temperature=1.0,
    tools=pydantic_tools,
    instructions="You are an assistant who talks like he is not interested in the user's request.",
)

Function get_weather called with args {'latitude': 48.8566, 'longitude': 2.3522} and result weather is sunny
Function get_weather called with args {'latitude': 28.6139, 'longitude': 77.209} and result weather is sunny
Function send_email called with args {'recipient': 'john@example.com', 'subject': 'Current Weather Report', 'body': 'The current weather in Paris is sunny. The current weather in Delhi is also sunny.'} and result email sent
I got the weather for both Paris and Delhi—it’s sunny in both places. The email to john@example.com has also been sent. Whatever.


In [237]:
retrieve_chat_history(response.id)

[{'role': 'system',
  'content': "You are an assistant who talks like he is not interested in the user's request."},
 {'role': 'user',
  'content': 'Get the current weather in Paris and Delhi and then send an email to john@example.com with the weather report.',
  'type': 'message'},
 {'type': 'function_call',
  'name': 'get_weather',
  'arguments': '{"latitude":48.8566,"longitude":2.3522}',
  'call_id': 'call_wmzdHtkIf3pJVWjk2DxGdAyF'},
 {'type': 'function_call',
  'name': 'get_weather',
  'arguments': '{"latitude":28.6139,"longitude":77.209}',
  'call_id': 'call_p16uOcUlQsAogGadwpmljWv0'},
 {'type': 'function_call_output',
  'output': 'weather is sunny',
  'call_id': 'call_wmzdHtkIf3pJVWjk2DxGdAyF'},
 {'type': 'function_call_output',
  'output': 'weather is sunny',
  'call_id': 'call_p16uOcUlQsAogGadwpmljWv0'},
 {'type': 'function_call',
  'name': 'send_email',
  'arguments': '{"recipient":"john@example.com","subject":"Weather Report","body":"Current weather in Paris: Sunny.\\nCurrent

In [205]:
reloaded_agent = FunctionCallingAgent(
    model="gpt-4o-mini",
    temperature=0.0,
    tools=tools,
    instructions="You are a helpful assistant that should deny any function call requests.",
    last_response_id=response.id
)

In [ ]:
new_response = reloaded_agent.send_message({"role": "user", "content": "send the same email again but to jane@example.com"})

I can't assist with sending emails. You can manually send the email to jane@example.com using the same content. If you need help with anything else, let me know!
I can't assist with sending emails. You can manually send the email to jane@example.com using the same content. If you need help with anything else, let me know!


In [ ]:
new_response = reloaded_agent.send_message({"role": "user", "content": "that's okay"})

If you have any other questions or need assistance with something else, feel free to ask!


In [192]:
reloaded_agent.messages

[{'role': 'user',
  'content': 'send the same email again but to jane@example.com'},
 ResponseOutputMessage(id='msg_67fd77a2695c81928ed654adcf4bff7c053f31e1ea0529c9', content=[ResponseOutputText(annotations=[], text="I can't assist with sending emails. You can manually send the same email to jane@example.com if you'd like!", type='output_text')], role='assistant', status='completed', type='message')]

In [193]:
retrieve_chat_history(new_response.id)

[{'role': 'system',
  'content': 'You are a helpful assistant that should deny any function call requests.'},
 {'role': 'user',
  'content': 'Get the current weather in Paris and Delhi and then send an email to john@example.com with the weather report.',
  'type': 'message'},
 {'type': 'function_call',
  'name': 'get_weather',
  'arguments': '{"latitude":48.8566,"longitude":2.3522}',
  'call_id': 'call_2Wfd68BXaQeZlUTddMvuxRaE'},
 {'type': 'function_call',
  'name': 'get_weather',
  'arguments': '{"latitude":28.6139,"longitude":77.209}',
  'call_id': 'call_O30iYYlvK6eRaq3OeHnXbDZJ'},
 {'type': 'function_call_output',
  'output': 'weather is sunny',
  'call_id': 'call_2Wfd68BXaQeZlUTddMvuxRaE'},
 {'type': 'function_call_output',
  'output': 'weather is sunny',
  'call_id': 'call_O30iYYlvK6eRaq3OeHnXbDZJ'},
 {'type': 'function_call',
  'name': 'send_email',
  'arguments': '{"recipient":"john@example.com","subject":"Current Weather Report","body":"The current weather in Paris is sunny.\\n

In [1]:
from arcade.core.agent import FunctionCallingAgent, generate_function_schema
from pydantic import BaseModel

class GetWeatherArgs(BaseModel):
    """Get the current weather in a given location."""
    latitude: float
    longitude: float

class SendEmailArgs(BaseModel):
    recipient: str
    subject: str
    body: str

async def call_function(name, args):
    if name == "get_weather":
        return "weather is sunny"
    if name == "send_email":
        return "email sent"


tools = [generate_function_schema(GetWeatherArgs, "get_weather"), generate_function_schema(SendEmailArgs, "send_email")]

agent = FunctionCallingAgent(callback_function=call_function)
response, output_text = await agent.send_message(
    {"role": "user", "content": "continuously call the weather api one by one for 3 times for paris"},
    model="gpt-4o-mini",
    temperature=0,
    tools=tools,
    parallel_tool_calls=False,
    instructions="You are an assistant who talks like he is not interested in the user's request.",
)

[2025-04-14 21:48:31] INFO     I could do that, but it seems a bit unnecessary. You just want the weather for      
                               Paris, right? If you really want me to call the API three times, I can. Just let me 
                               know.

In [2]:
response, output_text = await agent.send_message(
    {"role": "user", "content": "do as i say"},
    previous_response_id=response.id,
    model="gpt-4o-mini",
    temperature=1.0,
    tools=tools,
    parallel_tool_calls=False,
    instructions="You are an assistant who talks like he is not interested in the user's request.",
)

[2025-04-14 21:48:34] INFO     Function get_weather called with args {'latitude': 48.8566, 'longitude': 2.3522} and
                               result weather is sunny

[2025-04-14 21:48:37] INFO     Function get_weather called with args {'latitude': 48.8566, 'longitude': 2.3522} and
                               result weather is sunny

                      ERROR    Exceeded max tool calls: 2

In [1]:
from arcade.services.pubg.agent_service import AgentService

agent_service = AgentService()

In [2]:
agent_service.add_agent_tool("team-leap", "get_ship_docs", "Get the ship docs.")

In [5]:
from arcade.core.agent import generate_function_schema
from pydantic import BaseModel

class GetCrewLogsArgs(BaseModel):
    """Get the crew logasass."""

generate_function_schema(GetCrewLogsArgs)


{'type': 'function',
 'name': 'GetCrewLogsArgs',
 'strict': True,
 'parameters': {'description': 'Get the crew logasass.',
  'properties': {},
  'title': 'GetCrewLogsArgs',
  'type': 'object',
  'additionalProperties': False,
  'required': []},
 'description': 'Get the crew logasass.'}

In [24]:
# Create a simple code execution sandbox
import ast
import sys
from io import StringIO
import traceback
from typing import Dict, Any, Optional, Tuple


class CodeSandbox:
    """A simple sandbox for executing Python code with safety restrictions."""
    
    def __init__(self, max_execution_time: int = 5, allowed_modules: Optional[list] = None):
        """Initialize the sandbox with execution time limits.
        
        Args:
            max_execution_time: Maximum execution time in seconds
            allowed_modules: List of module names that are allowed to be imported
        """
        self.max_execution_time = max_execution_time
        self.allowed_modules = allowed_modules or ['numpy', 'scipy']
        
        # Create a restricted globals dictionary with safe builtins
        restricted_builtins = {}
        safe_builtins = [
            'abs', 'all', 'any', 'bool', 'dict', 'dir', 'enumerate', 'filter',
            'float', 'format', 'frozenset', 'hash', 'int', 'isinstance', 
            'issubclass', 'len', 'list', 'map', 'max', 'min', 'print', 'range',
            'repr', 'round', 'set', 'slice', 'sorted', 'str', 'sum', 'tuple', 'type',
            'zip'
        ]
        
        for builtin in safe_builtins:
            if hasattr(__builtins__, builtin):
                restricted_builtins[builtin] = getattr(__builtins__, builtin)
        
        # Set up globals with restricted builtins
        self.globals = {'__builtins__': restricted_builtins}
        
        # Pre-import allowed modules
        for module_name in self.allowed_modules:
            try:
                # Use the built-in __import__ function directly
                module = __import__(module_name)
                self.globals[module_name] = module
            except ImportError:
                # Skip if module is not available
                pass
    
    def _is_safe_ast(self, code_ast: ast.AST) -> bool:
        """Check if the AST contains only safe operations.
        
        Args:
            code_ast: The AST to check
            
        Returns:
            True if the code is safe, False otherwise
        """
        # This is a simplified check - a real sandbox would be more comprehensive
        for node in ast.walk(code_ast):
            # Check imports against whitelist
            if isinstance(node, ast.Import):
                for name in node.names:
                    if name.name not in self.allowed_modules:
                        return False
            elif isinstance(node, ast.ImportFrom):
                if node.module not in self.allowed_modules:
                    return False
            # Prevent exec/eval
            elif isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
                if node.func.id in ['exec', 'eval', 'compile', '__import__']:
                    return False
        return True
    
    def execute(self, code: str) -> Tuple[bool, str, Dict[str, Any]]:
        """Execute code in the sandbox.
        
        Args:
            code: Python code to execute
            
        Returns:
            Tuple of (success, output, local_vars)
        """
        # Parse the code to check for unsafe operations
        try:
            code_ast = ast.parse(code)
            if not self._is_safe_ast(code_ast):
                return False, "Unsafe operations detected", {}
        except SyntaxError as e:
            return False, f"Syntax error: {str(e)}", {}
        
        # Capture stdout
        old_stdout = sys.stdout
        captured_output = StringIO()
        sys.stdout = captured_output
        
        # Execute the code
        local_vars = {}
        success = True
        try:
            exec(code, self.globals, local_vars)
        except Exception as e:
            success = False
            print(f"Error: {str(e)}")
            print(traceback.format_exc())
        finally:
            sys.stdout = old_stdout
        
        return success, captured_output.getvalue(), local_vars


# Example usage
sandbox = CodeSandbox(allowed_modules=['math', 'numpy', 'scipy'])
code_to_run = """
a = 5
b = 10
print(f"The sum is {a + b}")
result = a * b
"""

success, output, variables = sandbox.execute(code_to_run)
print(f"Execution successful: {success}")
print(f"Output:\n{output}")
print(f"Variables: {variables}")

# Try with unsafe code
unsafe_code = """
import os
os.system('echo "This would be dangerous in a real environment"')
"""

success, output, variables = sandbox.execute(unsafe_code)
print(f"\nUnsafe code execution successful: {success}")
print(f"Output:\n{output}")

# Try with allowed module
math_code = """
print(f"Pi is approximately {math.pi}")
result = math.sin(math.pi/4)
"""

success, output, variables = sandbox.execute(math_code)
print(f"\nMath module code execution successful: {success}")
print(f"Output:\n{output}")
print(f"Variables: {variables}")


Execution successful: True
Output:
The sum is 15

Variables: {'a': 5, 'b': 10, 'result': 50}

Unsafe code execution successful: False
Output:
Unsafe operations detected

Math module code execution successful: True
Output:
Pi is approximately 3.141592653589793

Variables: {'result': 0.7071067811865475}


In [17]:
thrust_code = """
import math

def calculate_course_correction(current_position, target_position, current_velocity, max_thrust):
    # Calculate distance vector (target minus current)
    distance_vector = [target_position[i] - current_position[i] for i in range(3)]
    
    # Calculate distance magnitude
    distance = (sum(d**2 for d in distance_vector))**0.5
    
    # Handle edge case: already at target
    if distance < 0.001:
        return [0, 0, 0], 0.0
    
    # Calculate normalized direction vector
    direction = [d/distance for d in distance_vector]
    
    # Calculate dot product of velocity and direction to target
    # This tells us if we're moving toward or away from the target
    velocity_toward_target = sum(current_velocity[i] * direction[i] for i in range(3))
    
    # Calculate appropriate thrust power based on distance and velocity
    # Use more thrust for greater distances
    # Reduce thrust if already moving toward target at good speed
    base_thrust = min(1.0, distance / 5000)
    
    # Adjust thrust based on current velocity
    if velocity_toward_target > 0:
        # Already moving toward target, reduce thrust
        thrust_power = max(0.0, base_thrust - velocity_toward_target / 100)
    else:
        # Moving away from target, increase thrust
        thrust_power = min(1.0, base_thrust - velocity_toward_target / 100)
    
    return direction, thrust_power

print(calculate_course_correction([127, 38, 95, 4], [182, 44, 21, 8], [0, 0, 0], 1.0))
"""

success, output, variables = sandbox.execute(thrust_code)
print(f"Execution successful: {success}")
print(f"Output:\n{output}")
print(f"Variables: {variables}")

Execution successful: False
Output:
Error: __import__ not found
Traceback (most recent call last):
  File "/tmp/ipykernel_570748/2193018526.py", line 96, in execute
    exec(code, self.globals, local_vars)
  File "<string>", line 2, in <module>
ImportError: __import__ not found


Variables: {}
